# <p align=center> NBA Games

## Sumary

## About

In this project, we are going to create an NBA games database from scraping data from the [flashscore](https://www.flashscore.com/basketball/usa/nba/) website.

## Setting up the environment

In [22]:
import bs4
import helium
import selenium
import sqlite3
import time

import queries
import scripts

In [14]:
options = selenium.webdriver.FirefoxOptions()
options.add_argument("--width=1024")
options.add_argument("--height=768")
options.add_argument("-headless")

## Creating the sources database

In [7]:
year = 2020

In [10]:
path = f'data/sources/sources_{year}.db'
with sqlite3.connect(path) as con:
    query = queries.CREATE_TABLE_SOURCES
    cur = con.cursor()
    cur.execute(query)

In [11]:
url = f'flashscore.com/basketball/usa/nba-{year}-{year+1}/results/'
with helium.start_firefox() as driver:
    helium.go_to(url)
    time.sleep(2)
    scripts.scroll_results_page(driver)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')

In [21]:
ids = soup.find_all('div', {'class': 'event__match'})
ids = [id_['id'] for id_ in ids]
ids = [id_.split('_')[-1] for id_ in ids]

ids[:5]

['ULD28jPK', 'zBYhhpYl', '4nnd7v16', 'Qemh8Ko0', 'rXcm90Wg']

In [ ]:
def get_values_db_sources(id_, driver):
    keys = [
        'summary_source',
        'stats_source',
    ]
    values = {key: None for key in keys}
    values['id'] = id_
    url = f'flashscore.com/match/{id_}/'

    href_summary = '#match-summary'
    helium.go_to(f'{url}{href_summary}')
    time.sleep(2)
    soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
    soup = soup.find('div', {'class': 'container__detail'})
    values['summary_source'] = str(soup)

    href_stats = '#match-summary/match-statistics'
    if soup.find_all('a', {'href': href_stats}):
        helium.go_to(f'{url}{href_stats}')
        time.sleep(2)
        soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
        soup = soup.find('div', {'class': 'container__detail'})
        values['stats_source'] = str(soup)

    return values

In [ ]:
with helium.start_firefox(options=options) as driver:
    for id_ in ids:
        values = get_values_db_sources(id_, driver)        
        with sqlite3.connect(f'data/sources/sources_{year}.db') as con:
            query = queries.INSERT_INTO_SOURCES
            cur = con.cursor()
            cur.execute(query, values)

## Creating the games database

## Author